In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [11]:
import os
KEY=os.getenv("OPENAI_API_KEY")

In [14]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [23]:
loader = DirectoryLoader(
    "new_articles", 
    glob="*.txt", 
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)

In [24]:
document=loader.load()

In [ ]:
document

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text = text_splitter.split_documents(document)

In [ ]:
text[0].page_content

In [ ]:
text[1].page_content

In [ ]:
len(text)

# Creating DB

In [31]:
from langchain import embeddings

In [32]:
persist_directory = 'db'

In [ ]:
embedding = OpenAIEmbeddings()

In [ ]:
vectordb = Chroma.from_documents(documents=text, embedding=embedding, persist_directory=persist_directory)

In [ ]:
# persiste the DB to disk
vectordb.persist()

In [ ]:
vectordb=None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
vectordb

### Make a retriver

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("How much money did Microsoft raise?")

In [ ]:
print(docs[0].page_content)

In [ ]:
len(docs)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
retriever.search_kwargs

In [ ]:
docs2 = retriever.get_relevant_documents("How much money did Microsoft raise?")

In [ ]:
len(docs2)

In [ ]:
docs2

### Make a Chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
llm=OpenAI()

In [ ]:
llm

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "How much money did Microsoft raise?"

In [ ]:
llm_response = qa_chain(query)

In [ ]:
llm_response

In [ ]:
process_llm_response(llm_response)

### Delete the DB

In [ ]:
!zip -r db.zip ./db

In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

In [ ]:
# delete the directory
!rm -rf db/